In [ ]:
import random

import chained_filter
import fourier_flow_generator
import garch_generator
import gen_data_loader
import illiquidity_filter
import infty_filter
import len_filter
import numpy as np
import pandas as pd
import real_data_loader
import time_filter
import torch
import visualizer
from arch.univariate import StudentsT

## Set seeds for reconstruction of the results

In [ ]:
# Set seeds.
SEED = 12345
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)

## Load Data

Here the data gets generated / prepared for the later analysis

#### Prepare Filters

In [ ]:
inf_filter = infty_filter.InftyFilter()
iliq_filter = illiquidity_filter.IlliquidityFilter(
    window=20, min_jumps=2, exclude_tolerance=10
)
l_filter = len_filter.LenFilter(min_points=4000)

time_filt = time_filter.TimeFilter(pd.Timestamp(year=1990, month=1, day=1))

filter = chained_filter.ChainedFilter(
    [inf_filter, iliq_filter, l_filter], time_filter=time_filt
)
filter.reset_filter()

#### Load Data

In [ ]:
real_loader = real_data_loader.RealDataLoader(cache="../data/cache")
real_data = real_loader.get_timeseries(
    col_name="Adj Close", data_path="../data/raw_yahoo_data", update_all=False
)
# fit filter for real data
filter.fit_filter(real_data)

In [ ]:
garch = garch_generator.GarchGenerator(
    p=3, q=3, distribution=StudentsT(), name="GARCH_3_3_student"
)
data_loader = gen_data_loader.GenDataLoader(cache="../data/cache")
gen_data = data_loader.get_timeseries(
    generator=garch, col_name="Adj Close", data_loader=real_loader, update_all=False
)
# filter garch data as well
filter.fit_filter(gen_data)

In [ ]:
ff_gen = fourier_flow_generator.FourierFlowGenerator(
    learning_rate=1e-2, epochs=1000, dtype=torch.float32
)
ff_loader = gen_data_loader.GenDataLoader(cache="../data/cache")
ff_data = ff_loader.get_timeseries(
    ff_gen,
    data_loader=real_loader,
    col_name="Adj Close",
    update_all=False,
    symbols=["MSFT", "AMZN"],
)

In [ ]:
iliq_filter.print_state()
l_filter.print_state()
filter.print_state()

#### Apply Filters

In [ ]:
filter.apply_filter(gen_data)
filter.apply_filter(real_data)

### Generate Plots

In [ ]:
visualizer.visualize_all(
    ff_data.loc[:, ["MSFT", "AMZN"]], name="Fourier Flow Data MSFT and AMZN"
).figure.show()

In [ ]:
visualizer.visualize_all(
    real_data.loc[:, ["MSFT", "AMZN"]], name="Real Data MSFT and AMZN"
).figure.show()

In [ ]:
visualizer.visualize_all(
    gen_data.loc[:, ["MSFT", "AMZN"]], name=garch.name
).figure.show()